# inport data

# read DATA


In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

In [5]:
file_path = "../data/filled_data_label_numeric.xlsx"
filled_data_label_numeric_df = pd.read_excel(file_path,engine='openpyxl')
filled_data_label_numeric_df

,gender,dept_flg,area_flg,cntr_flg,agent_level,agent_status,fyp_month_avg,persistence_prem_25,salary_ded_ratio,xagfd_flag,...,rule30_counts_missing_flag,rule31_counts_missing_flag,rule32_counts_missing_flag,rule33_counts_missing_flag,rule34_counts_missing_flag,rule35_counts_missing_flag,rule36_counts_missing_flag,rule37_counts_missing_flag,rule38_counts_missing_flag,rule39_counts_missing_flag
0,1,2,4,4,25,2,30054.25,94.42,33.33,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,9,3,11,0,68389.32,100.00,33.33,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,4,4,33,1,14618.46,94.42,33.33,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2,4,28,33,1,42400.30,94.42,33.33,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2,4,28,33,0,23869.50,0.00,33.33,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4014,0,2,4,4,33,1,30268.00,94.42,33.33,0,...,0,0,0,0,0,0,0,0,0,0
4015,0,2,5,8,33,1,44524.35,94.42,33.33,0,...,0,0,0,0,0,0,0,0,0,0
4016,1,2,4,28,27,1,36144.52,94.42,33.33,0,...,0,0,0,0,0,0,0,0,0,0
4017,0,1,13,5,33,1,30268.00,94.42,33.33,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# 刪除非數值型欄位，例如 abnormal_target 及標記缺失值的欄位
columns_to_drop = ["abnormal_target"] + [col for col in filled_data_label_numeric_df.columns if "missing_flag" in col]
df_cleaned = filled_data_label_numeric_df.drop(columns=columns_to_drop)


In [15]:
df_cleaned

,gender,dept_flg,area_flg,cntr_flg,agent_level,agent_status,fyp_month_avg,persistence_prem_25,salary_ded_ratio,xagfd_flag,xagdt_flag,age,register_until_now,bill_invalid_ratio,bill_invalid_cnt,agent_own_plcy,one_year_claim_plcy_cnt,one_year_claim_plcy_ratio,addr_not_vld_rate,comm_amount_1y_rate,pc_in_60_days_rate,lapse_2y_rate,cusp_own_flag,cusp_intr_flag,rule1_counts,rule2_counts,rule3_counts,rule4_counts,rule5_counts,rule6_counts,rule7_counts,rule8_counts,rule9_counts,rule10_counts,rule11_counts,rule12_counts,rule13_counts,rule14_counts,rule15_counts,rule16_counts,rule17_counts,rule18_counts,rule19_counts,rule20_counts,rule21_counts,rule22_counts,rule23_counts,rule24_counts,rule25_counts,rule26_counts,rule27_counts,rule28_counts,rule29_counts,rule30_counts,rule31_counts,rule32_counts,rule33_counts,rule34_counts,rule35_counts,rule36_counts,rule37_counts,rule38_counts,rule39_counts
0,1,2,4,4,25,2,30054.25,94.42,33.33,0,1,32,11,0.02,1.0,4,1,0.02,0.00,0.20,0.75,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,9,3,11,0,68389.32,100.00,33.33,0,1,34,6,0.13,14.0,11,6,0.03,0.03,0.20,0.46,0.09,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,3,0,0,1,0,0,0
2,1,2,4,4,33,1,14618.46,94.42,33.33,0,0,33,9,0.02,1.0,4,1,0.02,0.00,0.20,0.75,0.00,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,1,2,4,28,33,1,42400.30,94.42,33.33,0,1,40,18,0.02,1.0,4,1,0.02,0.00,0.20,0.75,0.00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,2,4,28,33,0,23869.50,0.00,33.33,0,1,25,1,0.00,0.0,3,0,0.00,0.09,0.19,0.82,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4014,0,2,4,4,33,1,30268.00,94.42,33.33,0,0,67,18,0.02,1.0,4,1,0.02,0.00,0.20,0.75,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4015,0,2,5,8,33,1,44524.35,94.42,33.33,0,0,69,25,0.02,1.0,4,1,0.02,0.00,0.20,0.75,0.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,1,0,0
4016,1,2,4,28,27,1,36144.52,94.42,33.33,0,1,45,15,0.02,1.0,4,1,0.02,0.00,0.20,0.75,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4017,0,1,13,5,33,1,30268.00,94.42,33.33,0,0,59,22,0.02,1.0,4,1,0.02,0.00,0.20,0.75,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [16]:
from sklearn.preprocessing import StandardScaler

# 對數據進行標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_cleaned)


# 做PCA

In [17]:
from sklearn.decomposition import PCA

# 設定要保留的主成分數量，這裡設置為所有主成分
pca = PCA(n_components=min(X_scaled.shape[0], X_scaled.shape[1]))  # 保留所有主成分
X_pca = pca.fit_transform(X_scaled)

# 查看每個主成分的解釋變異量
print("解釋變異量：", pca.explained_variance_ratio_)


解釋變異量： [0.12676236 0.06042261 0.04203637 0.03470778 0.03301742 0.0295038
 0.02533601 0.02366412 0.02152197 0.0210929  0.02057944 0.0200627
 0.01964615 0.01904141 0.01888645 0.01790755 0.0177693  0.01714118
 0.0169398  0.01677611 0.01635182 0.015699   0.01551573 0.01535069
 0.01480826 0.01455141 0.01427522 0.01385343 0.01358191 0.01329369
 0.01299179 0.01272692 0.01232526 0.01185162 0.01163227 0.01142252
 0.0112359  0.0109222  0.01061103 0.01048063 0.01034162 0.00995644
 0.0096221  0.00934956 0.00914527 0.00854607 0.00832691 0.0081971
 0.00775044 0.00762161 0.00730889 0.00677539 0.006185   0.00563644
 0.00489766 0.00442434 0.00343424 0.00266727 0.00253993 0.00084927
 0.00012774 0.         0.        ]


做XGBoost

In [21]:
# 使用PCA後的數據進行訓練
X_pca_df = pd.DataFrame(X_pca)  # 將PCA後的數據轉換為DataFrame

# 分割數據為訓練集和測試集（80%訓練，20%測試）
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca_df, filled_data_label_numeric_df["abnormal_target"], test_size=0.2, random_state=42)

# 使用 SMOTE 進行過採樣
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_pca, y_train)

# 計算過採樣後的正負樣本比例，並設置 scale_pos_weight
scale_pos_weight_resampled = (y_train_resampled.value_counts()[0] / y_train_resampled.value_counts()[1])

# 設定 XGBoost 的參數
params = {
    'objective': 'binary:logistic',  # 二元分類
    'eval_metric': 'logloss',        # 損失函數
    'max_depth': 6,                  # 樹的最大深度
    'learning_rate': 0.1,            # 學習率
    'n_estimators': 100,             # 迭代次數
    'use_label_encoder': False,      # 禁用 label encoder
    'early_stopping_rounds': 10,     # 早停，防止過度擬合
    'scale_pos_weight': scale_pos_weight_resampled,  # 設置權重來應對不平衡資料
}

# 建立模型
model = xgb.XGBClassifier(**params)

# 訓練模型，並提供驗證集進行早停
eval_set = [(X_test_pca, y_test)]
model.fit(X_train_resampled, y_train_resampled, eval_set=eval_set, verbose=True)


[0]	validation_0-logloss:0.60041
[1]	validation_0-logloss:0.52461
[2]	validation_0-logloss:0.46159
[3]	validation_0-logloss:0.40842
[4]	validation_0-logloss:0.36310
[5]	validation_0-logloss:0.32403
[6]	validation_0-logloss:0.29035
[7]	validation_0-logloss:0.26094
[8]	validation_0-logloss:0.23512
[9]	validation_0-logloss:0.21266
[10]	validation_0-logloss:0.19272
[11]	validation_0-logloss:0.17534
[12]	validation_0-logloss:0.15966
[13]	validation_0-logloss:0.14587
[14]	validation_0-logloss:0.13368
[15]	validation_0-logloss:0.12287
[16]	validation_0-logloss:0.11311
[17]	validation_0-logloss:0.10466
[18]	validation_0-logloss:0.09701
[19]	validation_0-logloss:0.09022
[20]	validation_0-logloss:0.08352
[21]	validation_0-logloss:0.07751
[22]	validation_0-logloss:0.07206
[23]	validation_0-logloss:0.06721
[24]	validation_0-logloss:0.06263
[25]	validation_0-logloss:0.05866
[26]	validation_0-logloss:0.05515
[27]	validation_0-logloss:0.05187
[28]	validation_0-logloss:0.04895
[29]	validation_0-loglos

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\callback.py:386: UserWarning: [09:43:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

檢查結果

In [22]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 用測試集進行預測
y_pred = model.predict(X_test_pca)

# 計算準確率
accuracy = accuracy_score(y_test, y_pred)
print(f"準確率: {accuracy:.4f}")

# 顯示分類報告
print("分類報告:")
print(classification_report(y_test, y_pred))

# 顯示混淆矩陣
print("混淆矩陣:")
print(confusion_matrix(y_test, y_pred))


準確率: 0.9938
分類報告:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       795
           1       0.64      1.00      0.78         9

    accuracy                           0.99       804
   macro avg       0.82      1.00      0.89       804
weighted avg       1.00      0.99      0.99       804

混淆矩陣:
[[790   5]
 [  0   9]]


目前可以看到混淆矩陣中被誤認的部分5特別明顯，接下來應該可以透過調整參數處理這個部分